Добрый день! меня зовут Вильданова Регина и ниже выполнение моего тестового задания на позицию junior-аналитик.    


В задании указано использовать SQL ддля выполнения. Хочу пояснить, что в своей работе  применяю python и sql в паре. Анализировать данные на python для меня проще и приятнее, потому что в курсе Яндекса был акцент именно на него. Но и sql я не упускаю из набора своих навыков: тренирую, изучаю дополнительно, чтобы овладеть им в достаточной мере для оперативного решения задач. Упражняюсь на https://sql-ex.ru/, начала книгу Дж.Селко "Стиль программирования на SQL", прослушала курс Глеба Михайлова по SQL.  

В целом мне хотелось бы не ограничиваться определенным инструметом, а применять различного рода методы, которые конкретно могут решить задачу оперативно и корректно.

# Какой тариф выгоднее продавать?

[Ссылка на Excel-таблицу](kakoy_tarif_vigodnee_prodavat.xlsx)

#  Анализ датасета для дашборда

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
pd.set_option("display.max_columns", 100)

In [3]:
import sqlite3

In [4]:
con = sqlite3.connect('db')

In [5]:
df = pd.read_csv('anyclass_FF_sales.csv')

###  Общая информация

In [6]:
#первые 5 строк датасета для общего представления
df.head()

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1380260,29808,1140909,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24,2023-07-24,2023-03-02,NaN,RUB,Cloud Sber ff-anyclass.com,netherlands,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,1357778.0,refund
1,1380410,97673,1104802,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24,2023-07-24,2023-01-11,NaN,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1366729.0,refund
2,1367755,355586,1102857,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-12,2023-07-12,2023-01-09,NaN,RUB,Cloud Sber ff-anyclass.com,uzbekistan,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1364654.0,refund
3,1380241,538009,1106637,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24,2023-07-24,2023-01-13,NaN,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1369240.0,refund
4,1387048,532216,1238628,FF,online,148,SUPER Лицо. Перезагрузка,2023-07-30,2023-07-30,2023-07-30,NaN,RUB,Cloud Sber ff-anyclass.com,russia,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,NaN,income


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31545 entries, 0 to 31544
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   invoice_id                  31545 non-null  int64  
 1   user_id                     31545 non-null  int64  
 2   order_id                    31545 non-null  int64  
 3   school                      31545 non-null  object 
 4   product                     31545 non-null  object 
 5   course_id                   31545 non-null  int64  
 6   course_title                31545 non-null  object 
 7   paid_date                   31545 non-null  object 
 8   invoice_created_date        31545 non-null  object 
 9   order_created_date          31545 non-null  object 
 10  subscription_created_date   31125 non-null  object 
 11  currency                    31545 non-null  object 
 12  payment_system              31545 non-null  object 
 13  country                     315

В датасете 26 столбцов и 31545 строк, типы данных не соотвествуют в столбцах paid_date, invoice_created_date, order_created_date, subscription_created_date, есть пропуски.

## Типы данных.

In [8]:
#Приведем данные столбцы к типу datetime. 
df[['paid_date', 'invoice_created_date', 'order_created_date', 'subscription_created_date']] = \
df[['paid_date', 'invoice_created_date', 'order_created_date', 'subscription_created_date']].\
apply(pd.to_datetime)

In [9]:
#проверка типов
#df.info()

## Пропуски.

In [10]:
#кол-во пропущенных значений 
df.isna().sum()

invoice_id                        0
user_id                           0
order_id                          0
school                            0
product                           0
course_id                         0
course_title                      0
paid_date                         0
invoice_created_date              0
order_created_date                0
subscription_created_date       420
currency                          0
payment_system                    0
country                           0
paid_count                        0
paid_amount                       0
first_paid_count                  0
first_paid_amount                 0
manual_created_count              0
manual_created_amount             0
extend_subscription_count         0
extend_subscription_amount        0
subscription_period             420
rank                              0
parent_invoice_id             31240
type                              0
dtype: int64

В датасете 420 пропусков в subscription_created_date(дата старта подписки) и subscription_period (период подписки), т.е. 420   invoice_id это заказы без подписки.    
305 пропусков в parent_invoice_id         

## Дубликаты.

In [11]:
df.duplicated().sum()

0

Явных дубликатов нет.  
Проверим на неявные дубликаты как опечатки столбец course_title, country.

In [12]:
#наименования курсов 
sorted(df['course_title'].unique())

['NEW SUPER Лицо',
 'SUPER Лицо PRO',
 'SUPER Лицо ТЕСТ',
 'SUPER Лицо. Перезагрузка',
 'Super Тело: йога дома',
 'Женский клуб Я/Мы SUPER Лицо',
 'Мьюинг для всех',
 'Прямые эфиры c Анастасией Бурдюг',
 'Секреты SUPER Лица']

In [13]:
df.pivot_table(index = ['product','course_id', 'course_title'], values = 'paid_amount', aggfunc ='sum')

paid_amount
product course_id course_title                                   
club    62        SUPER Лицо PRO                       117.306122
        148       SUPER Лицо. Перезагрузка             111.020408
        188       Женский клуб Я/Мы SUPER Лицо       48198.227752
extend  62        SUPER Лицо PRO                         4.010204
        163       Прямые эфиры c Анастасией Бурдюг       3.820408
main    121       SUPER Лицо ТЕСТ                        0.005102
        148       SUPER Лицо. Перезагрузка          177426.155404
        207       NEW SUPER Лицо                     11994.194071
muing   171       Мьюинг для всех                       72.522092
offline 46        Секреты SUPER Лица                  1926.428571
online  148       SUPER Лицо. Перезагрузка            1055.419643
pro     62        SUPER Лицо PRO                        22.441327
yoga    172       Super Тело: йога дома                  8.066327

ООО,а курсы это у нас на естественное омоложение, респект! отдаю дань уважения, и в том числе Анастасии Бурдюг за пропаганду этого направления в массах.    

У нас есть уникальный id только для курса и он не меняется в зависимости от продукта.  

РЕКОМЕНДАЦИЯ: изменить id курсов с привязкой к типу продукта. 


In [14]:
#страны
df['country'].nunique()

130

Представлено 130 стран. Посмотрим какие.

In [15]:
sorted(df['country'].unique())


['-',
 'albania',
 'algeria',
 'andorra',
 'antigua & barbuda',
 'argentina',
 'armenia',
 'australia',
 'austria',
 'azerbaijan',
 'bahrain',
 'bangladesh',
 'belarus',
 'belgium',
 'belize',
 'bosnia & herzegovina',
 'brazil',
 'bulgaria',
 'cambodia',
 'canada',
 'chile',
 'china',
 'colombia',
 'congo - kinshasa',
 'costa rica',
 'croatia',
 'cyprus',
 'czechia',
 'côte d’ivoire',
 'denmark',
 'dominican republic',
 'ecuador',
 'egypt',
 'estonia',
 'finland',
 'france',
 'french guiana',
 'french southern territories',
 'georgia',
 'germany',
 'ghana',
 'gibraltar',
 'greece',
 'guadeloupe',
 'guernsey',
 'guinea',
 'hong kong',
 'hungary',
 'iceland',
 'india',
 'indonesia',
 'iraq',
 'ireland',
 'israel',
 'italy',
 'jamaica',
 'japan',
 'jordan',
 'kazakhstan',
 'kenya',
 'kuwait',
 'kyrgyzstan',
 'laos',
 'latvia',
 'lebanon',
 'lithuania',
 'luxembourg',
 'macao',
 'malaysia',
 'maldives',
 'mali',
 'malta',
 'mauritania',
 'mauritius',
 'mexico',
 'moldova',
 'monaco',
 'mon

Вижу,   
что есть неявные дубликаты: отдельно выделены регионы Франции 'french guiana', 'french southern territories','guadeloupe','réunion' - Франция, 'gibraltar', ireland, 'turks & caicos islands' - территории Великобритании, 'hong kong', 'macao' - территории Китая,'åland islands' - Финляндия.   
Скорректируем страны, заменим значения. 

РЕКОМЕНДАЦИЯ: настроить кодировку страны программистами или если страна вводиться вручную, сделать выпадающий список.

In [16]:
# Франция 
replace_values = ['french guiana', 'french southern territories', 'guadeloupe','réunion']
for i in replace_values:
    df['country'] = df['country'].replace(i, "france")
    
#Великобритания   
replace_values = ['gibraltar', 'turks & caicos islands', 'ireland']
for i in replace_values:
    df['country'] = df['country'].replace(i, "united kingdom")
    
#Китай   
replace_values = ['hong kong', 'macao']
for i in replace_values:
    df['country'] = df['country'].replace(i, "china")
    
#Финляндия   
replace_values = ['åland islands']
for i in replace_values:
    df['country'] = df['country'].replace(i, "finland")
    
    
df['country'].nunique()


120

Осталось 120 страна. 

## Дополнительно.


Проверю еще на дубликаты invoice_id, т.к. это уникальный номер операции и дубликатов здесь не должно быть. 

In [17]:
df['invoice_id'].nunique()

31545

Дубликатов в датасете не выявлено.  



# SQL


In [18]:
#записываем данные из таблицы в базу данных SQL
df.to_sql('anyclass_test',con,index=False,if_exists='replace')

31545

In [19]:
#функция для вывода sql-запроса
def select(sql):
  return pd.read_sql(sql,con)

## Общая информация

In [20]:
#обозрим нашу таблицу
sql = '''select * 
from anyclass_test t
limit 5'''
select(sql)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1380260,29808,1140909,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-03-02 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,netherlands,1,5.051020,1,5.051020,0,0.0,0,0.0,None,0,1357778.0,refund
1,1380410,97673,1104802,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-11 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,None,0,1366729.0,refund
2,1367755,355586,1102857,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-12 00:00:00,2023-07-12 00:00:00,2023-01-09 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,uzbekistan,1,13.622449,1,13.622449,0,0.0,0,0.0,None,0,1364654.0,refund
3,1380241,538009,1106637,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-13 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,None,0,1369240.0,refund
4,1387048,532216,1238628,FF,online,148,SUPER Лицо. Перезагрузка,2023-07-30 00:00:00,2023-07-30 00:00:00,2023-07-30 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,5.051020,1,5.051020,0,0.0,0,0.0,None,0,NaN,income


In [21]:
#названия столбцов списоком
sql = '''pragma table_info(anyclass_test)'''
select(sql)


,cid,name,type,notnull,dflt_value,pk
0,0,invoice_id,INTEGER,0,None,0
1,1,user_id,INTEGER,0,None,0
2,2,order_id,INTEGER,0,None,0
3,3,school,TEXT,0,None,0
4,4,product,TEXT,0,None,0
5,5,course_id,INTEGER,0,None,0
6,6,course_title,TEXT,0,None,0
7,7,paid_date,TIMESTAMP,0,None,0
8,8,invoice_created_date,TIMESTAMP,0,None,0
9,9,order_created_date,TIMESTAMP,0,None,0


В описании не указаны invoice_id,user_id, order_id, но в принципе эти столбцы интуитивно понятны. В остально описание данных соотвествует.

In [22]:
#кол-во строк
sql = '''select count(*)
from anyclass_test t
'''
select(sql)

,count(*)
0,31545


Все ок, 26 столбцов и 31545 строк.  
Посмотрим какая информация в каждом столбце и сразу в разрезе заказы-возвраты

## Определение продуктов/курсов.

Посмотрим сколько услуг представлено в датасете 

In [23]:
#наименование школ
sql = '''
select count(distinct(school)) as cnt_sch,
count(distinct(product)) as cnt_prod,
count(distinct(course_id)) as cnt_c_id,
count(distinct(course_title)) as cnt_c_title
from anyclass_test t
'''
select(sql)

,cnt_sch,cnt_prod,cnt_c_id,cnt_c_title
0,1,8,9,9


Итак, у нас 1 школа, 8 типов продуктов и 9 наименований курсов.  
Разверенем подробнее. 

In [24]:
#кол-во заказов и возвратов по продуктам
sql = '''
with inv as -- заказы
        (select distinct (product),  --страна
        count(*) as cnt_inv, -- кол-во инвойсов
        round (sum(paid_amount),2) as sum_paid_inv 
        from anyclass_test t
        group by product
        order by sum(paid_amount) desc),

     ref as --возраты
        (select distinct (product),  
        count(*) as cnt_ref, -- кол-во возратов
        round(sum(paid_amount),2) as sum_paid_ref 
        from anyclass_test t
        where type = 'refund'
        group by product
        order by sum(paid_amount) desc)

select inv.product,
inv.cnt_inv*1.0 as cnt_inv, -- кол-во заказов 
ref.cnt_ref, -- кол-во возвратов 
round ((ref.cnt_ref/(inv.cnt_inv*1.0)*100), 4) as per_cnt_ref, -- процент вовратов по кол-ву 
inv.sum_paid_inv, -- сумма заказов 
ref.sum_paid_ref, -- кол-во заказов 
round ((ref.sum_paid_ref/(inv.sum_paid_inv*1.0)*100), 4) as per_sum_ref -- процент возратов по сумме
from inv
left join ref 
on inv.product = ref.product
order by cnt_ref desc
limit 40


'''
select(sql)

,product,cnt_inv,cnt_ref,per_cnt_ref,sum_paid_inv,sum_paid_ref,per_sum_ref
0,main,25164.0,285.0,1.1326,189420.35,3041.59,1.6057
1,club,6285.0,31.0,0.4932,48426.55,353.58,0.7301
2,offline,42.0,NaN,NaN,1926.43,NaN,NaN
3,online,33.0,NaN,NaN,1055.42,NaN,NaN
4,muing,12.0,NaN,NaN,72.52,NaN,NaN
5,pro,6.0,NaN,NaN,22.44,NaN,NaN
6,yoga,1.0,NaN,NaN,8.07,NaN,NaN
7,extend,2.0,NaN,NaN,7.83,NaN,NaN


Возвраты были в продуктах club и main. Отобразим эти же данные в разрезе курсов.

In [25]:
#кол-во заказов и возвратов по продуктам
sql = '''
with inv as
        (select product,
        course_title, 
        count(*) as cnt_inv, -- кол-во инвойсов
        round (sum(paid_amount),2) as sum_paid_inv -- сумма транзакций по валюте в евро
        from anyclass_test t
        group by product, course_title
        order by sum(paid_amount) desc),

     ref as 
        (select  product,
        course_title,  
        count(*) as cnt_ref, -- кол-во возратов
        round(sum(paid_amount),2) as sum_paid_ref -- сумма транзакций по валюте в евро
        from anyclass_test t
        where type = 'refund'
        group by product, course_title
        order by sum(paid_amount) desc)

select inv.product,
inv.course_title,
inv.cnt_inv*1.0 as cnt_inv,
ref.cnt_ref,
round ((ref.cnt_ref/(inv.cnt_inv*1.0)*100), 4) as per_cnt_ref,
inv.sum_paid_inv,
ref.sum_paid_ref,
round ((ref.sum_paid_ref/(inv.sum_paid_inv*1.0)*100), 4) as per_sum_ref
from inv
left join ref 
on inv.course_title = ref.course_title and inv.product = ref.product
order by cnt_ref desc
limit 40


'''
select(sql)

,product,course_title,cnt_inv,cnt_ref,per_cnt_ref,sum_paid_inv,sum_paid_ref,per_sum_ref
0,main,SUPER Лицо. Перезагрузка,23382.0,285.0,1.2189,177426.16,3041.59,1.7143
1,club,Женский клуб Я/Мы SUPER Лицо,6240.0,31.0,0.4968,48198.23,353.58,0.7336
2,main,NEW SUPER Лицо,1781.0,NaN,NaN,11994.19,NaN,NaN
3,offline,Секреты SUPER Лица,42.0,NaN,NaN,1926.43,NaN,NaN
4,online,SUPER Лицо. Перезагрузка,33.0,NaN,NaN,1055.42,NaN,NaN
5,club,SUPER Лицо PRO,21.0,NaN,NaN,117.31,NaN,NaN
6,club,SUPER Лицо. Перезагрузка,24.0,NaN,NaN,111.02,NaN,NaN
7,muing,Мьюинг для всех,12.0,NaN,NaN,72.52,NaN,NaN
8,pro,SUPER Лицо PRO,6.0,NaN,NaN,22.44,NaN,NaN
9,yoga,Super Тело: йога дома,1.0,NaN,NaN,8.07,NaN,NaN


Лидерами и по кол-ву прихода денег и по кол-ву операций являются  ЖенскийклубЯ/МыSUPERЛицо (club) и  SUPERЛицо.Перезагрузка(main).
Что там за строки где только 1 операция c копеечными платами?

In [26]:
#данные с 1 оплатой 
sql = '''
select  course_id,
course_title, 
product, 
count(*) as cnt_act, --кол-во транзакций на курсе
round(sum(paid_amount),2) as sum_paid,--получено оплат в евро
*
from anyclass_test t
group by course_title, product
having cnt_act =1
order by sum_paid desc
'''
select(sql)

,course_id,course_title,product,cnt_act,sum_paid,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,172,Super Тело: йога дома,yoga,1,8.07,1388550,793936,1239430,FF,yoga,172,Super Тело: йога дома,2023-07-31 00:00:00,2023-07-31 00:00:00,2023-07-31 00:00:00,None,EUR,Ipay,moldova,1,8.066327,1,8.066327,0,0.000000,0,0.0,None,0,None,income
1,62,SUPER Лицо PRO,extend,1,4.01,1366037,52861,1137157,FF,extend,62,SUPER Лицо PRO,2023-07-10 00:00:00,2023-07-10 00:00:00,2023-02-25 00:00:00,None,RUB,sber anyclass,united states,1,4.010204,1,4.010204,1,4.010204,0,0.0,None,0,None,income
2,163,Прямые эфиры c Анастасией Бурдюг,extend,1,3.82,1335564,555301,1210972,FF,extend,163,Прямые эфиры c Анастасией Бурдюг,2023-06-14 00:00:00,2023-06-14 00:00:00,2023-06-14 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,3.820408,1,3.820408,0,0.000000,0,0.0,None,0,None,income
3,121,SUPER Лицо ТЕСТ,main,1,0.01,1381283,792945,1235539,FF,main,121,SUPER Лицо ТЕСТ,2023-07-25 00:00:00,2023-07-25 00:00:00,2023-07-25 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,0.005102,1,0.005102,1,0.005102,0,0.0,None,0,None,income


Все это заказы прихода, возратов нет. С курсом SUPER Лицо ТЕСТ(main) все понятно, тестовая, смело удаляем. Остальные незнаю, может тоже тестовые для платежных систем, но почему разные суммы тогда? Могу предположить,что Super Тело: йога дома только стартовала 31,.07.2021 г., а данные в датасете только по 31.07.2023, поэтому только 1 оплата. В любом случае, у нас только 1 оплата по этим курсам и нет возратов, данные настолько малы, что не могут повлиять на общую картину. Уберем их для итогового датасета.

In [27]:
#удаляем данные с 1 оплатой
sql = '''
    select  *
    from anyclass_test t
    where invoice_id not in (select  invoice_id
    from anyclass_test t
    group by course_title, product
    having count(*) =1)
    
'''
select(sql)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1380260,29808,1140909,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-03-02 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,netherlands,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,1357778.0,refund
1,1380410,97673,1104802,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-11 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1366729.0,refund
2,1367755,355586,1102857,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-12 00:00:00,2023-07-12 00:00:00,2023-01-09 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,uzbekistan,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1364654.0,refund
3,1380241,538009,1106637,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-13 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1369240.0,refund
4,1387048,532216,1238628,FF,online,148,SUPER Лицо. Перезагрузка,2023-07-30 00:00:00,2023-07-30 00:00:00,2023-07-30 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,NaN,income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31536,1349816,775567,1219167,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,EUR,Stripe Romania,united arab emirates,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31537,1349496,784095,1218976,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,EUR,Stripe Romania,greece,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31538,1350161,734859,1219370,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-26 00:00:00,2023-06-26 00:00:00,2023-06-26 00:00:00,2023-06-26 00:00:00,EUR,Stripe Romania,lithuania,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31539,1351825,776520,1220301,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-27 00:00:00,2023-06-27 00:00:00,2023-06-27 00:00:00,2023-06-27 00:00:00,EUR,Stripe Romania,united states,1,3.913265,1,3.913265,0,0.0,0,0.0,1.0,1,NaN,income


In [28]:
#записываем скорретированный датасет в result и далее работаем с ним
result = select(sql)
#записываем данные из таблицы в базу данных SQL
result.to_sql('result',con,index=False,if_exists='replace')

31541

In [29]:
# #проверка, что строки корректно удалились 
# sql = '''
# select  course_id,
# course_title, 
# product, 
# count(*) as cnt_act, --кол-во транзакций на курсе
# round(sum(paid_amount),2) as sum_paid,--получено оплат в евро
# *
# from result 
# group by course_title, product
# having cnt_act =1
# order by sum_paid desc
# '''
# select(sql)

## Срезы дат.

Посмотрим за какой период у нас есть данные.

In [30]:
#период оплат

sql = '''
select min (paid_date), 
max (paid_date),
min (invoice_created_date),
max (invoice_created_date),
min (order_created_date),
max (order_created_date),
min (subscription_created_date),
max (subscription_created_date)
from result
'''
select(sql)

,min (paid_date),max (paid_date),min (invoice_created_date),max (invoice_created_date),min (order_created_date),max (order_created_date),min (subscription_created_date),max (subscription_created_date)
0,2023-06-01 00:00:00,2023-07-31 00:00:00,2021-04-30 00:00:00,2023-08-15 00:00:00,2021-04-30 00:00:00,2023-07-31 00:00:00,2022-08-30 00:00:00,2023-07-31 00:00:00


Датасет содержит   
оплаты с 1 июня по 31 июля 2023г.,  
платежки создавались с 30 апреля 2021г. по 15 августа 2023г.,    
заказы регистрировались с 30 апреля 2021г. по 31 июля 2023г.,  
подписки стартовали с 30 августа 2022г. по 31 июля 2023г.   

Почему в данных такая разница между платежка, заказами и оплатами. Ладно подписки - ну куда ни шло начинаются со второй половины 2022г., но платежки и заказы аж с 2021г.?Были предзаказы, которые завершились оплатой только в 2023г.?  
Возможно ли, что пользователь долго  присматривался к курсу, начал оформление, и решился на покупку только спустя 2 года? 
Посмотрим сколько у нас данных по годам и что в них.

In [31]:
#срез по датам создания платежки за 2021-2022 г. 
sql = '''
select *
from result 
where strftime('%Y', invoice_created_date)  in ('2021', '2022') 
'''
select(sql)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,581137,458282,615490,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-21 00:00:00,2021-04-30 00:00:00,2021-04-30 00:00:00,None,RUB,Cloud anyclass.com,-,1,6.071429,1,6.071429,0,0.0,0,0.0,None,0,None,income


Итак, у нас одна оплата с платежкой от 2021г. Трудно представляю, что платежку начали создавать в 2021г, а оплату провели в 2023г., делаю вывод - что это ошибка в данных платежки.

In [32]:
#срез по датам создания заказов за 2021-2022 г. 
sql = '''
select *
from result
where strftime('%Y', order_created_date)  in ('2021', '2022')
order by order_created_date
'''
select(sql)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,581137,458282,615490,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-21 00:00:00,2021-04-30 00:00:00,2021-04-30 00:00:00,None,RUB,Cloud anyclass.com,-,1,6.071429,1,6.071429,0,0.0,0,0.000000,NaN,0,None,income
1,1385776,104052,670518,FF,pro,62,SUPER Лицо PRO,2023-07-29 00:00:00,2023-07-29 00:00:00,2021-07-03 00:00:00,None,RUB,sber anyclass,russia,1,7.091837,1,7.091837,0,0.0,0,0.000000,NaN,0,None,income
2,1383135,167774,1022520,FF,pro,62,SUPER Лицо PRO,2023-07-26 00:00:00,2023-07-26 00:00:00,2022-08-30 00:00:00,2022-08-30 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,4.540816,0,0.000000,0,0.0,1,4.540816,1.0,5,None,income
3,1350857,167774,1022520,FF,pro,62,SUPER Лицо PRO,2023-06-26 00:00:00,2023-06-26 00:00:00,2022-08-30 00:00:00,2022-08-30 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,4.540816,0,0.000000,0,0.0,1,4.540816,1.0,4,None,income
4,1334588,680457,1024921,FF,club,62,SUPER Лицо PRO,2023-06-13 00:00:00,2023-06-13 00:00:00,2022-09-03 00:00:00,2022-09-04 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,8.112245,0,0.000000,0,0.0,1,8.112245,3.0,2,None,income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536,1354642,717977,1094909,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-30 00:00:00,2023-06-30 00:00:00,2022-12-31 00:00:00,2022-12-31 00:00:00,RUB,Cloud anyclass.com,russia,1,13.622449,0,0.000000,0,0.0,1,13.622449,6.0,2,None,income
537,1388572,558599,1095212,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-31 00:00:00,2023-07-31 00:00:00,2022-12-31 00:00:00,2022-12-31 00:00:00,RUB,Cloud anyclass.com,russia,1,4.540816,0,0.000000,0,0.0,1,4.540816,1.0,8,None,income
538,1388216,337295,1094974,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-31 00:00:00,2023-07-31 00:00:00,2022-12-31 00:00:00,2022-12-31 00:00:00,RUB,Cloud Sber ff-anyclass.com,spain,1,4.540816,0,0.000000,0,0.0,1,4.540816,1.0,8,None,income
539,1388790,42323,1095337,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-31 00:00:00,2023-07-31 00:00:00,2022-12-31 00:00:00,2022-12-31 00:00:00,RUB,Cloud Sber ff-anyclass.com,thailand,1,4.540816,0,0.000000,0,0.0,1,4.540816,1.0,8,None,income


541 строка! С датой создания заказа уже не все так явно.   
Учитывая объем данных, ладно, допускаю, что пользователь "зрел" до покупки так долго.   
Такое несоотвествие имеет право на более детальное ислледование, но в данной работе для анализа будут взяты только даты оплаты.  Так как важно показать ситуацию по возвратам, а возврат (как и заказ) можно считать осуществленнным только при проведении оплаты.  Посмотрим в каком временнном промежутке у нас были вовзраты. 

In [33]:
#периоды для возвратов

sql = '''
select min (paid_date), 
max (paid_date),
min (invoice_created_date),
max (invoice_created_date),
min (order_created_date),
max (order_created_date),
min (subscription_created_date),
max (subscription_created_date)
from result
where type='refund'
'''
select(sql)

,min (paid_date),max (paid_date),min (invoice_created_date),max (invoice_created_date),min (order_created_date),max (order_created_date),min (subscription_created_date),max (subscription_created_date)
0,2023-06-06 00:00:00,2023-07-24 00:00:00,2023-06-06 00:00:00,2023-07-24 00:00:00,2023-01-03 00:00:00,2023-07-19 00:00:00,2023-06-06 00:00:00,2023-06-15 00:00:00


Все возвратные заказы, платежки и оплаты  были в 2023 году, ок, этого достаточно. Сразу возникает вопрос почему указывается подписка для возвратов? Но тоже пока не буду в это углубляться, т.к. напомню для нас важна именно проведение оплаты по возврату. 

## Срезы валюты,страны.

### Валюта

In [34]:
#кол-во заказов и возвратов по валютам
sql = '''
with inv as
        (select distinct (currency),  --платежная система
        count(*) as cnt_inv, -- кол-во инвойсов
        round (sum(paid_amount),2) as sum_paid_inv -- сумма транзакций по валюте в евро
        from result
        group by currency
        order by sum(paid_amount) desc),

    ref as 
        (select distinct (currency),  
        count(*) as cnt_ref, -- кол-во инвойсов
        round(sum(paid_amount),2) as sum_paid_ref -- сумма транзакций по валюте в евро
        from result
        where type = 'refund'
        group by currency
        order by sum(paid_amount) desc)

select inv.currency,
inv.cnt_inv*1.0 as cnt_inv,
ref.cnt_ref,
round ((ref.cnt_ref/(inv.cnt_inv*1.0)*100), 4) as per_ref,
inv.sum_paid_inv,
ref.sum_paid_ref,
round ((ref.sum_paid_ref/(inv.sum_paid_inv*1.0)*100), 4) as per_sum_ref
from inv
left join ref 
on inv.currency = ref.currency


'''
select(sql)

,currency,cnt_inv,cnt_ref,per_ref,sum_paid_inv,sum_paid_ref,per_sum_ref
0,RUB,19976.0,266.0,1.3316,149297.56,2292.57,1.5356
1,EUR,8554.0,27.0,0.3156,64588.32,896.86,1.3886
2,KZT,2067.0,23.0,1.1127,18389.50,205.74,1.1188
3,BYN,297.0,NaN,NaN,2776.17,NaN,NaN
4,ILS,135.0,NaN,NaN,1151.62,NaN,NaN
5,KGS,119.0,NaN,NaN,1127.19,NaN,NaN
6,MDL,109.0,NaN,NaN,1092.15,NaN,NaN
7,UZS,101.0,NaN,NaN,979.18,NaN,NaN
8,UAH,122.0,NaN,NaN,956.84,NaN,NaN
9,AZN,54.0,NaN,NaN,515.71,NaN,NaN


Оплату проводят 11 валютами (дубликатов нет), больше всего в рублях, евро и тенге. И именно в этих валютах и были возраты.

### Платежные системы.

In [35]:
#кол-во заказов и возвратов по платежным системам
sql = '''
with ps as
        (select distinct (payment_system),  --платежная система
        count(*) as cnt_inv, -- кол-во инвойсов
        round (sum(paid_amount),2) as sum_paid_inv -- сумма транзакций по валюте в евро
        from result
        group by payment_system
        order by sum(paid_amount) desc),

    ps_refund as 
        (select distinct (payment_system),  
        count(*) as cnt_ref, -- кол-во возвратов
        round(sum(paid_amount),2) as sum_paid_ref -- сумма транзакций по валюте в евро
        from result
        where type = 'refund'
        group by payment_system
        order by sum(paid_amount) desc)

select ps.payment_system,
ps.cnt_inv*1.0 as cnt_inv,
ps_refund.cnt_ref,
round ((ps_refund.cnt_ref/(ps.cnt_inv*1.0)*100), 4) as per_cnt_ref,
ps.sum_paid_inv,
ps_refund.sum_paid_ref,
round ((ps_refund.sum_paid_ref/(ps.sum_paid_inv*1.0)*100), 4) as per_sum_ref
from ps
left join ps_refund 
on ps.payment_system = ps_refund.payment_system
order by cnt_ref desc


'''
select(sql)

,payment_system,cnt_inv,cnt_ref,per_cnt_ref,sum_paid_inv,sum_paid_ref,per_sum_ref
0,Cloud Sber ff-anyclass.com,19012.0,258.0,1.3570,140802.96,2230.99,1.5845
1,Stripe Romania,8115.0,42.0,0.5176,64054.03,1016.40,1.5868
2,Cloud anyclass.com,616.0,8.0,1.2987,4283.55,61.58,1.4376
3,PayPal Romania,1184.0,4.0,0.3378,8694.07,37.26,0.4286
4,Stripe Cyprus НЕ ИСПОЛЬЗОВАТЬ,197.0,3.0,1.5228,1279.32,40.32,3.1517
5,NEW Stripe Romania,1955.0,1.0,0.0512,16783.86,8.62,0.0514
6,sber anyclass,306.0,NaN,NaN,2316.47,NaN,NaN
7,Cloud facetraining.ru ИП Серебряков,41.0,NaN,NaN,1885.66,NaN,NaN
8,Ipay,76.0,NaN,NaN,578.16,NaN,NaN
9,PayPal Cyprus,35.0,NaN,NaN,225.08,NaN,NaN


Платежи проходят через 11 платежных систем.  
Больше всего платежей проходит через системы Cloud Sber ff-anyclass.com,Stripe Romania,NEW Stripe Romania.  
Что случилось с Stripe Romania, почему есть NEW Stripe Romania? также есть пс, которая помечена как "не использовать", зачем?
Могут ли возвраты быть связаны с техническими проблемами в платежных системах? (сделать вкладку в дашборде по платежным системам)

### Страны.

In [36]:
#кол-во заказов и возвратов по платежным системам
sql = '''
with inv as
        (select distinct (country),  --страна
        count(*) as cnt_inv, -- кол-во инвойсов
        round (sum(paid_amount),2) as sum_paid_inv -- сумма транзакций по валюте в евро
        from result
        group by country
        order by sum(paid_amount) desc),

    ref as 
        (select distinct (country),  
        count(*) as cnt_ref, -- кол-во возратов
        round(sum(paid_amount),2) as sum_paid_ref -- сумма транзакций по валюте в евро
        from result
        where type = 'refund'
        group by country
        order by sum(paid_amount) desc)

select inv.country,
inv.cnt_inv*1.0 as cnt_inv,
ref.cnt_ref,
round ((ref.cnt_ref/(inv.cnt_inv*1.0)*100), 4) as per_cnt_ref,
inv.sum_paid_inv,
ref.sum_paid_ref,
round ((ref.sum_paid_ref/(inv.sum_paid_inv*1.0)*100), 4) as per_sum_ref
from inv
left join ref 
on inv.country = ref.country
order by cnt_ref desc
limit 40


'''
select(sql)

,country,cnt_inv,cnt_ref,per_cnt_ref,sum_paid_inv,sum_paid_ref,per_sum_ref
0,russia,10889.0,126.0,1.1571,80278.11,1102.57,1.3734
1,germany,3029.0,28.0,0.9244,22918.08,246.75,1.0767
2,france,1766.0,28.0,1.5855,13076.73,214.47,1.6401
3,kazakhstan,2363.0,26.0,1.1003,20688.79,223.85,1.0820
4,netherlands,1051.0,15.0,1.4272,8909.55,807.11,9.0589
5,united states,1461.0,9.0,0.6160,11212.64,69.44,0.6193
6,united kingdom,801.0,9.0,1.1236,6160.12,87.85,1.4261
7,spain,492.0,7.0,1.4228,3826.13,42.62,1.1139
8,belarus,1002.0,6.0,0.5988,7802.88,54.08,0.6931
9,türkiye,590.0,6.0,1.0169,4452.84,50.71,1.1388


121 страна и в 39 из них прошли возвраты. Хоть возвраты у нас только в трех валютах, возвраты не только из России, Казахстана и Европы.   
Если ранее лидеры были сформированы по процентному соотношению вовзратов, то в случае со странами сделана сортировка по кол-ву возратов. Например, Саудовская Аравия имеет высокий % возратов, но это из-за того что там всего 16 заказов и возрат сразу делает большой минус. 	


## Срезы покупок.

### paid_count (1 - по ивойсу проведен один платеж любого типа)

In [37]:
#посмотрим какие значения у нас в paid_count и сразу посчитаем кол-во
sql = '''
select paid_count, 
type,
count (*), 
round(sum (paid_amount),2) as sum_paid
from result
group by paid_count, type

'''
select(sql)

,paid_count,type,count (*),sum_paid
0,0,income,425,282.90
1,0,refund,11,100.34
2,1,income,30800,237245.65
3,1,refund,305,3294.83


Посмотрим, что это за 11 отдельно стоящих возвратов. 

In [38]:
sql = '''
select *
from result
where type ='refund' and paid_count = 0

'''
select(sql)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1328415,732770,1206846,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,8.923469,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
1,1327654,561407,1206450,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,RUB,Cloud Sber ff-anyclass.com,united kingdom,0,8.923469,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
2,1326615,700693,1205880,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,15.255102,0,0.0,0,0.0,0,0.0,6.0,1,None,refund
3,1330644,779570,1208162,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,8.622449,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
4,1331501,109719,1208716,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,8.622449,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
5,1329908,779374,1207655,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-09 00:00:00,2023-06-09 00:00:00,2023-06-09 00:00:00,2023-06-09 00:00:00,RUB,Cloud Sber ff-anyclass.com,netherlands,0,8.622449,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
6,1327083,778607,1206139,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,5.051020,0,0.0,0,0.0,0,0.0,1.0,1,None,refund
7,1336818,781130,1211756,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,9.132653,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
8,1336299,640373,1211445,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,9.132653,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
9,1336738,661480,1211701,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,0,9.132653,0,0.0,0,0.0,0,0.0,3.0,1,None,refund


У всех этих 11 возратов есть период подписки, почему? странно что у возрата есть период подписки. Все заказы не были созданы вргучную. Или это возраты по подписке, а остальная возраты были при единоразовых платежах.   
Почему у нас указано paid_count= 0  для заказов и возратов,где была сумма оплаты?  Нелогично, исправим это. 

In [39]:
#заменим paid_count = 1 где есть оплата с помощью pandas
result.loc [result['paid_amount']>0, 'paid_count' ] = 1

In [40]:
#записываем данные из таблицы в базу данных SQL
result.to_sql('result',con,index=False,if_exists='replace')

31541

In [41]:
# #проверка
# #посмотрим какие значения у нас в paid_count и сразу посчитаем кол-во
# sql = '''
# select paid_count, 
# type,
# count (*), 
# round(sum (paid_amount),2) as sum_paid
# from result
# group by paid_count, type

# '''
# select(sql)

### first_paid_count (1 - по инвойсу проведен один первичный платеж в подписке или платеж по единоразовым покупкам)

In [42]:
#посмотрим какие значения у нас в first_paid_count и сразу посчитаем кол-во по заказам и возратам
sql = '''
select first_paid_count, 
type,
count (*),
round(sum (first_paid_amount),2) as sum_paid
from result
group by first_paid_count, type

'''
select(sql)

,first_paid_count,type,count (*),sum_paid
0,0,income,14431,0.00
1,0,refund,11,0.00
2,1,income,16794,138522.67
3,1,refund,305,3294.83


Подтверждается, что было 11 возратов по подписке. 

### manual_created_count (1 - заказ создан вручную отделом продаж)

In [43]:
#посмотрим какие значения у нас в manual_created_count и сразу посчиатем кол-во
sql = '''
select manual_created_count, 
type,
count (*),
round(sum (manual_created_amount),2) as sum_ca
from result
group by manual_created_count, type

'''
select(sql)

,manual_created_count,type,count (*),sum_ca
0,0,income,30162,0.00
1,0,refund,300,0.00
2,1,income,1063,9342.68
3,1,refund,16,139.75


Итак, есть возможность создавать вручную и продажу и возврат.  Всего 16 возратов создано вручную, 300 автоматом. Возможно были технические нюансы и платеж автоматически возращался?

### extend_subscription_count (1 - по инвойсу проведено одно продление)

In [44]:
sql = '''
select distinct (extend_subscription_count),
type,
count(*)
from result t
group by extend_subscription_count, type
'''
select(sql)

,extend_subscription_count,type,count(*)
0,0,income,17219
1,0,refund,316
2,1,income,14006


Логично, что по возрату не было продления, здесь все ок.

## Доппараметры.

### Количество месяцев в тарифе

In [45]:
sql = '''
select subscription_period,
type,
count(*)
from result t
group by subscription_period, type
'''
select(sql)

,subscription_period,type,count(*)
0,NaN,income,111
1,NaN,refund,305
2,1.0,income,11486
3,1.0,refund,1
4,2.0,income,5
5,3.0,income,17726
6,3.0,refund,9
7,6.0,income,1514
8,6.0,refund,1
9,7.0,income,145


И снова 11 возратов по подписке, больше всего на 3мес. подписке - 9шт.

### rank - очередность оплаты в подписке

In [46]:
sql = '''
select rank, type
,count(*) as cnt
from result t
group by rank, type
'''
select(sql)

,rank,type,cnt
0,-1,refund,3
1,0,income,495
2,0,refund,302
3,1,income,16695
4,1,refund,11
5,2,income,10795
6,3,income,1579
7,4,income,507
8,5,income,353
9,6,income,316


302 возврата при единоразовой покупке  и 11 при подписке. Runk - 1 что значит?

### parent_invoice_id - ID исходного платежа, который присваивается платежам типа refund для прямой связи прихода и возврата

In [47]:
#выведем parent_invoice_id, проверим нет ли дубликатов, увидим общее кол-во 
sql = '''
select parent_invoice_id
,count(*) as cnt
from result t
group by parent_invoice_id
order by cnt desc

'''
select(sql)

,parent_invoice_id,cnt
0,NaN,31236
1,1380013.0,1
2,1379555.0,1
3,1379549.0,1
4,1379109.0,1
...,...,...
301,1341032.0,1
302,1331430.0,1
303,1309935.0,1
304,1247497.0,1


Почему возратных инвойсов только 305? на 11 заказов нет возвратных инвойсов. Посмотрим эти строки

In [48]:
sql = '''
select *
from result t
where parent_invoice_id  is  null and type = 'refund' -- не присвоено id возврату

'''
select(sql)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1328415,732770,1206846,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,8.923469,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
1,1327654,561407,1206450,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,2023-06-07 00:00:00,RUB,Cloud Sber ff-anyclass.com,united kingdom,1,8.923469,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
2,1326615,700693,1205880,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,15.255102,0,0.0,0,0.0,0,0.0,6.0,1,None,refund
3,1330644,779570,1208162,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,8.622449,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
4,1331501,109719,1208716,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,2023-06-10 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,8.622449,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
5,1329908,779374,1207655,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-09 00:00:00,2023-06-09 00:00:00,2023-06-09 00:00:00,2023-06-09 00:00:00,RUB,Cloud Sber ff-anyclass.com,netherlands,1,8.622449,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
6,1327083,778607,1206139,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,2023-06-06 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,5.051020,0,0.0,0,0.0,0,0.0,1.0,1,None,refund
7,1336818,781130,1211756,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,9.132653,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
8,1336299,640373,1211445,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,9.132653,0,0.0,0,0.0,0,0.0,3.0,1,None,refund
9,1336738,661480,1211701,FF,main,148,SUPER Лицо. Перезагрузка,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,2023-06-15 00:00:00,RUB,Cloud Sber ff-anyclass.com,russia,1,9.132653,0,0.0,0,0.0,0,0.0,3.0,1,None,refund


Это те же заказы, что были обнаружены при анализе подписок. Они созданы автоматом, а не отделом продаж, опять таки подтверждение, что это скорее тех.баг. Связать их с корреным заказом никак, удалим их поэтому.


In [49]:
sql = '''
select *
from result 
where invoice_id  not in (select invoice_id 
from result t
where parent_invoice_id  is  null and type = 'refund') -- номера заказов не присвоено id возврату

'''
select(sql)

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1380260,29808,1140909,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-03-02 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,netherlands,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,1357778.0,refund
1,1380410,97673,1104802,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-11 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1366729.0,refund
2,1367755,355586,1102857,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-12 00:00:00,2023-07-12 00:00:00,2023-01-09 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,uzbekistan,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1364654.0,refund
3,1380241,538009,1106637,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-13 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1369240.0,refund
4,1387048,532216,1238628,FF,online,148,SUPER Лицо. Перезагрузка,2023-07-30 00:00:00,2023-07-30 00:00:00,2023-07-30 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,NaN,income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31525,1349816,775567,1219167,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,EUR,Stripe Romania,united arab emirates,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31526,1349496,784095,1218976,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,EUR,Stripe Romania,greece,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31527,1350161,734859,1219370,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-26 00:00:00,2023-06-26 00:00:00,2023-06-26 00:00:00,2023-06-26 00:00:00,EUR,Stripe Romania,lithuania,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31528,1351825,776520,1220301,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-27 00:00:00,2023-06-27 00:00:00,2023-06-27 00:00:00,2023-06-27 00:00:00,EUR,Stripe Romania,united states,1,3.913265,1,3.913265,0,0.0,0,0.0,1.0,1,NaN,income


In [50]:
result=select(sql)

In [51]:
#записываем данные из таблицы в базу данных SQL
result.to_sql('result',con,index=False,if_exists='replace')

31530

### type - тип инвойса

In [52]:
sql = '''
select type,
count(*) as cnt, 
round (sum (paid_amount),2) as sum_paid
from result t
group by type

'''
select(sql)

,type,cnt,sum_paid
0,income,31225,237528.55
1,refund,305,3294.83


Здесь все ок.

## Итоговая таблица для построения дашборда. 

Сперва свяжем заказы и возраты,это можно сделать сопоставив invoice_id и parent_invoice_id.

In [53]:
# #запишем в result
# result=select(sql)
# #записываем данные из таблицы в базу данных SQL
# result.to_sql('result',con,index=False,if_exists='replace')

In [54]:
result

,invoice_id,user_id,order_id,school,product,course_id,course_title,paid_date,invoice_created_date,order_created_date,subscription_created_date,currency,payment_system,country,paid_count,paid_amount,first_paid_count,first_paid_amount,manual_created_count,manual_created_amount,extend_subscription_count,extend_subscription_amount,subscription_period,rank,parent_invoice_id,type
0,1380260,29808,1140909,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-03-02 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,netherlands,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,1357778.0,refund
1,1380410,97673,1104802,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-11 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1366729.0,refund
2,1367755,355586,1102857,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-12 00:00:00,2023-07-12 00:00:00,2023-01-09 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,uzbekistan,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1364654.0,refund
3,1380241,538009,1106637,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-07-24 00:00:00,2023-07-24 00:00:00,2023-01-13 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,13.622449,1,13.622449,0,0.0,0,0.0,NaN,0,1369240.0,refund
4,1387048,532216,1238628,FF,online,148,SUPER Лицо. Перезагрузка,2023-07-30 00:00:00,2023-07-30 00:00:00,2023-07-30 00:00:00,None,RUB,Cloud Sber ff-anyclass.com,russia,1,5.051020,1,5.051020,0,0.0,0,0.0,NaN,0,NaN,income
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31525,1349816,775567,1219167,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,EUR,Stripe Romania,united arab emirates,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31526,1349496,784095,1218976,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,2023-06-25 00:00:00,EUR,Stripe Romania,greece,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31527,1350161,734859,1219370,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-26 00:00:00,2023-06-26 00:00:00,2023-06-26 00:00:00,2023-06-26 00:00:00,EUR,Stripe Romania,lithuania,1,14.030612,1,14.030612,0,0.0,0,0.0,6.0,1,NaN,income
31528,1351825,776520,1220301,FF,club,188,Женский клуб Я/Мы SUPER Лицо,2023-06-27 00:00:00,2023-06-27 00:00:00,2023-06-27 00:00:00,2023-06-27 00:00:00,EUR,Stripe Romania,united states,1,3.913265,1,3.913265,0,0.0,0,0.0,1.0,1,NaN,income


In [55]:
sql = '''
drop table if exists result_main;

create table result_main as 
select *
from result t
full outer join (select parent_invoice_id
from result ref 
where type='refund')   ref
on t.invoice_id = ref.parent_invoice_id
where t.invoice_id is not null
order by t.invoice_id

'''


In [56]:
cur = con.cursor()

In [57]:
cur.executescript(sql)

In [58]:
#посмотрим на получившуюся таблицу перед сохранением результатов
sql = '''
select * from result_main 
'''

In [59]:
#записываем данные в датафрейм
result_main=select(sql)

In [60]:
result_main.to_csv (r'out-result_main.csv', index= False )

# Дашборд
https://public.tableau.com/app/profile/regi.vil/viz/FFAnyClass/sheet4?publish=yes

Т.к. в процессе анализа я часто задавалась вопросом не возникают ли возвраты по тех. причине то поэтому взяла для анализа платежные системы и страны. И графики наглядно показали что проблема в платежной системе Cloud_Sber FF.  Старалась делать дашборд интутивно понятным, фильры по типу операции, датам,  странам,платежным системам. 